In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
data_dir = "/content/drive/MyDrive/comp448-hw3/dataset_4"

In [45]:
import torch
from torchvision import datasets, models, transforms
import copy
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import os
import numpy as np

In [46]:
#load pretrained alexNet
model_conv = models.alexnet(pretrained = True)

#freeze weights
for param in model_conv.parameters():
    param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [47]:
#see model arch
print(model_conv)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [48]:
no_classes = 3
#set the last layer, to be 3-classed
model_conv.classifier[6] = nn.Linear(model_conv.classifier[6].in_features, no_classes)

In [49]:
criterion = nn.CrossEntropyLoss() 
optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=1, gamma=0.1)

In [50]:
#switch to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_conv = model_conv.to(device) 

In [59]:
#resize for input and put to tensor the imgs
data_transforms = {
 'train': transforms.Compose([
    #in the paper, input size is 224
    transforms.Resize(224),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     #std=[0.229, 0.224, 0.225])
 ]),
 'val': transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     #std=[0.229, 0.224, 0.225])
   ]),                        
 'test': transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     #std=[0.229, 0.224, 0.225])
   ])
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
 data_transforms[x]) for x in ['train', 'val', 'test']}  

In [60]:
#Address class imbalance problem using weights
weights = {}
for x in ['train', 'val', 'test']:
    c_counts = np.zeros(3) 
    labels = image_datasets[x].targets

    for l in labels:
        c_counts[l] += 1
        
    weights[x] = np.zeros(len(labels))
    for i in range(len(labels)):
        weights[x][i] = len(labels) / (3.0 * c_counts[labels[i]])


sampler = {x: torch.utils.data.sampler.WeightedRandomSampler(weights[x], len(weights[x])) for x in ['train', 'val', 'test']}

In [61]:
def train_model(image_datasets, model, criterion, optimizer, scheduler, num_epochs):
  dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 4, #shuffle = True, 
                                                sampler = sampler[x], num_workers = 4 ) for x in ['train', 'val']}
  best_model_wts = copy.deepcopy(model.state_dict())
  best_no_corrects = 0
  for epoch in range(num_epochs):
    # Set the model to the training mode for updating the weights using 
    # the first portion of training images
    model.train() 
    for inputs, labels in dataloaders['train']: # iterate over data
      inputs = inputs.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      with torch.set_grad_enabled(True):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    # Set the model to the evaluation mode for selecting the best network 
    # based on the number of correctly classified validation images
    model.eval()
    no_corrects = 0
    for inputs, labels in dataloaders['val']:
      inputs = inputs.to(device)
      labels = labels.to(device)
      with torch.set_grad_enabled(False):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        no_corrects += torch.sum(preds == labels.data)
    if no_corrects > best_no_corrects:
      best_no_corrects = no_corrects
      best_model_wts = copy.deepcopy(model.state_dict())
    scheduler.step()
  # Load the weights of the best network
  model.load_state_dict(best_model_wts)
  return model

In [62]:
model = train_model(image_datasets, model_conv, criterion, optimizer_conv, exp_lr_scheduler, 100)

In [63]:
#performance
model.eval()
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 4, #shuffle = True, 
                                              sampler = sampler[x], num_workers = 4 ) for x in ['train', 'val', 'test']}

In [64]:
# Calculate Accuracy
with torch.no_grad():
    all = 0
    for x in ['train', 'val', 'test']:        
        conf_mat = torch.zeros(3, 3)

        for ins, labels in dataloaders[x] :
            ins = ins.to(device)
            labels = labels.to(device)
            outputs = model_conv(ins)
            _, preds = torch.max(outputs, 1)
            for i in range((np.prod(labels.size()))):
                conf_mat[labels[i]][preds[i]] += 1
        
        #look for diagonal to get true positives
        acc = conf_mat.diag()/conf_mat.sum(1)
        
        all = conf_mat.diag().sum() / conf_mat.sum()
        print(x, ":")
        print(conf_mat)
        print("Accuracy: ", acc)
        print("Overall Accuracy: ", all.item())
        print()


train :
tensor([[25.,  9., 28.],
        [ 0., 32., 30.],
        [ 0.,  0., 62.]])
Accuracy:  tensor([0.4032, 0.5161, 1.0000])
Overall Accuracy:  0.6397849321365356

val :
tensor([[ 3.,  1.,  7.],
        [ 0.,  9.,  2.],
        [ 0.,  0., 24.]])
Accuracy:  tensor([0.2727, 0.8182, 1.0000])
Overall Accuracy:  0.782608687877655

test :
tensor([[25.,  5., 18.],
        [ 0., 32., 18.],
        [ 0.,  0., 46.]])
Accuracy:  tensor([0.5208, 0.6400, 1.0000])
Overall Accuracy:  0.7152777910232544

